In [11]:
import docker
import uuid
import paramiko
import os
from timeit import default_timer as timer
from dataclasses import dataclass
import re
import requests

@dataclass
class HadoopRunResult:
    stdout: list[str]
    stderr: list[str]
    elapsed: str
        
def get_elapsed_time(res):
    def get_id_from_res(res):
        for line in res[1]:
            m = re.search('job_([0-9_]*)', line)
            if m != None and m.group(1) != '':
                return m.group(1)
        return None
    x = requests.get(f'http://resourcemanager:8088/ws/v1/cluster/apps/application_{get_id_from_res(res)}')
    return x.json()['app']['elapsedTime']

def run_in_master(command):
    ssh = paramiko.SSHClient()
    ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    ssh.connect("namenode", username="root", password="pass")
    ssh_stdin, ssh_stdout, ssh_stderr = ssh.exec_command(f"cd /app/ && . env_var.sh && {command}")
    return (ssh_stdout.readlines(), ssh_stderr.readlines())

def get_data_from_output_path(path):
    return f"{path}/merged.txt"

def print_hdfs_output(path):
    raw = run_in_master(f"hdfs dfs -cat {get_data_from_output_path(path)}")[0]
    print("\n".join(raw[0:1000]))

def merge_results(path):
    run_in_master(f"hdfs dfs -cat {path}/part-r-* | hdfs dfs -put - {path}/merged.txt")

# Wpływ replikacji danych

In [ ]:
covid_01_jar_path = "/data/master_volume/map_reduce_jars/covid_01.jar"
covid_01_input_path = "/datasets/covid-dataset.jsonl"
covid_01_output_path = "/out_covid_1" + str(uuid.uuid4())
_ = run_in_master(f"yarn jar {covid_01_jar_path} {covid_01_input_path} {covid_01_output_path}")
merge_results(covid_01_output_path)

In [12]:
def run_covid_02():
    covid_02_jar_path = "/data/master_volume/map_reduce_jars/covid_02.jar"
    covid_02_input_path = get_data_from_output_path(covid_01_output_path)
    covid_02_output_path = "/out_covid_2" + str(uuid.uuid4())
    res = run_in_master(f"yarn jar {covid_02_jar_path} {covid_02_input_path} {covid_02_output_path}")
    merge_results(covid_02_output_path)
    return HadoopRunResult(res[0], res[1], get_elapsed_time(res))

## 1 replika

In [13]:
_ = run_in_master("hdfs dfs -setrep -R 1 /")

In [16]:
results = []
for i in range(3):
    temp = run_covid_02();
    results.append(temp)

NameError: name 'run_covid_02' is not defined

In [15]:
print(a.elapsed)

119668


In [ ]:
## Czas

## 3 repliki

In [ ]:
_ = run_in_master("hdfs dfs -setrep -R 3 /")

In [ ]:
covid_01_jar_path = "/data/master_volume/map_reduce_jars/covid_01.jar"
covid_01_input_path = "/datasets/covid-dataset.jsonl"
covid_01_output_path = "/out_covid_1" + str(uuid.uuid4())

res = run_in_master(f"yarn jar {covid_01_jar_path} {covid_01_input_path} {covid_01_output_path}")
merge_results(covid_01_output_path)

In [ ]:
## Czas